In [1]:
# -*- coding: utf-8 -*-

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy

In [3]:
import os
import sys

## Handle the CV files

In [4]:
cv_path = os.path.join(os.getcwd(), 'data', 'sharepoint', 'cv')
cvs = os.listdir(cv_path)

In [5]:
def get_cv(i):
    return os.path.join(cv_path, cvs[i])

In [6]:
example = get_cv(0)
example

'C:\\Users\\tollef\\Documents\\Git\\nlp-intro\\data\\sharepoint\\cv\\albert danielsen.txt'

## Fetch and cleandata

In [7]:
# prepare spacy nlp object
spacy_model = 'nb_core_news_lg'
nlp = spacy.load(spacy_model, disable = ['parser', 'tagger', 'ner'])
nlp.add_pipe('sentencizer')

In [8]:
data = None
with open(example, 'r', encoding='utf-8') as f:
    data = f.readlines()

In [9]:
def get_tokens(text):
    skip_pattern = '\r\n \n\n \n\n\n!"-#$%&()--.*+,-./:;<=>?@[\\]^_`{|}~\t\n\r '
    return [token for token in text if token not in skip_pattern]

def clean_sent(sent):
    illegal = ['\n', '\t', '.', ':']
    def valid(tok):
        return (tok not in illegal) and tok.isalpha()
    
    return ' '.join([tok for tok in sent.split(' ') if valid(tok)])
    

# combine all data, use the spacy sentencizer
def get_sentences(data):
    sentences = [clean_sent(sent).strip() for sent in data]
    return '. '.join(sentences).strip()

In [10]:
text = get_sentences(data)
print(text)

Albert er en svilingeniør innen kybernetikk og utdannet ved med spesialisering innen embedded Han har opparbeidet seg bred erfaring innen programvareutvikling fra sine prosjekter hos tidligere arbeidsgivere og som. . Nylig har Albert jobbet i et prosjekt for Altera et stort med mål om å digitalisere arbeidsflyten ved modifikasjoner på skip i Hans bidrag i dette prosjektet består i utvikling av en responsiv og brukervennlig løsning i samt utvikling i Core i et domain driven Gjennomførelsen av prosjektet ble gjort ved bruk av Azure DevOps og agile. . Tidligere har Albert bidratt på et stort prosjekt for titulert APS av Personell tok sikte på å automatisere registrering og utdeling av sertifikater til sjøfolk som opererer i norske Løsningen ble utviklet med Azure Functions og message med en. . Alberts kompetanse inkluderer programmering av mikrokontrollere og design av hardware og kompetanse han har styrket ved praktisk gjennomføring av prosjekt og deltidsjobb for ved å utvikle hardware o

In [11]:
spacy_doc = nlp(text)
MIN_LEN = 25
sentences = [s.text.replace('.','').strip() for s in spacy_doc.sents if len(s) > MIN_LEN]
sentences

['Albert er en svilingeniør innen kybernetikk og utdannet ved med spesialisering innen embedded Han har opparbeidet seg bred erfaring innen programvareutvikling fra sine prosjekter hos tidligere arbeidsgivere og som',
 'Nylig har Albert jobbet i et prosjekt for Altera et stort med mål om å digitalisere arbeidsflyten ved modifikasjoner på skip i Hans bidrag i dette prosjektet består i utvikling av en responsiv og brukervennlig løsning i samt utvikling i Core i et domain driven Gjennomførelsen av prosjektet ble gjort ved bruk av Azure DevOps og agile',
 'Tidligere har Albert bidratt på et stort prosjekt for titulert APS av Personell tok sikte på å automatisere registrering og utdeling av sertifikater til sjøfolk som opererer i norske Løsningen ble utviklet med Azure Functions og message med en',
 'Alberts kompetanse inkluderer programmering av mikrokontrollere og design av hardware og kompetanse han har styrket ved praktisk gjennomføring av prosjekt og deltidsjobb for ved å utvikle hardw

## Process the data!

In [12]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

In [13]:
print(f'Length of text: {len(text)} characters')
print(text[:200] + '...')

Length of text: 4562 characters
Albert er en svilingeniør innen kybernetikk og utdannet ved med spesialisering innen embedded Han har opparbeidet seg bred erfaring innen programvareutvikling fra sine prosjekter hos tidligere arbeids...


In [14]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

52 unique characters


In [15]:
chars = tf.strings.unicode_split(sentences, input_encoding='UTF-8')
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab), mask_token=None)
ids = ids_from_chars(chars)

In [16]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
chars = chars_from_ids(ids)
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'Albert er en svilingeni\xc3\xb8r innen kybernetikk og utdannet ved med spesialisering innen embedded Han har opparbeidet seg bred erfaring innen programvareutvikling fra sine prosjekter hos tidligere arbeidsgivere og som',
       b'Nylig har Albert jobbet i et prosjekt for Altera et stort med m\xc3\xa5l om \xc3\xa5 digitalisere arbeidsflyten ved modifikasjoner p\xc3\xa5 skip i Hans bidrag i dette prosjektet best\xc3\xa5r i utvikling av en responsiv og brukervennlig l\xc3\xb8sning i samt utvikling i Core i et domain driven Gjennomf\xc3\xb8relsen av prosjektet ble gjort ved bruk av Azure DevOps og agile',
       b'Tidligere har Albert bidratt p\xc3\xa5 et stort prosjekt for titulert APS av Personell tok sikte p\xc3\xa5 \xc3\xa5 automatisere registrering og utdeling av sertifikater til sj\xc3\xb8folk som opererer i norske L\xc3\xb8sningen ble utviklet med Azure Functions og message med en',
       b'Alberts kompetanse inkluderer programmering av mikrokontrollere og design av hard

In [17]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [19]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

<tf.Tensor: shape=(4562,), dtype=int64, numpy=array([ 3, 36, 26, ..., 25, 43, 29], dtype=int64)>

In [25]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

BATCH_SIZE = 2
# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (sequences.map(split_input_target)
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

In [29]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024
class CvModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x
        
model = CvModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [33]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [34]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b' sj\xc3\xb8folk som opererer i norske L\xc3\xb8sningen ble utviklet med Azure Functions og message med en. . Alber'

Next Char Predictions:
 b' NxlnEVIAFNHPFtSvBnJuVriuzHKy\xc3\xb8LwSoS[UNK]tf\xc3\xb8IhbtkTh.b[UNK]oCP[UNK]uu.vx[UNK]d[UNK]Oib\xc3\xb8dggnEHk\xc3\xb8EOuxgdKoAaszFStFzkdFvvpsNCI'


In [36]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (2, 100, 53)  # (batch_size, sequence_length, vocab_size)
Mean loss:         3.96876


In [39]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [40]:
model.compile(optimizer='adam', loss=loss)

In [41]:
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
22/22 [==============================] - 16s 646ms/step - loss: 3.7537
Epoch 2/20
22/22 [==============================] - 14s 633ms/step - loss: 3.0089
Epoch 3/20
22/22 [==============================] - 14s 631ms/step - loss: 2.6928
Epoch 4/20
22/22 [==============================] - 14s 630ms/step - loss: 2.4601
Epoch 5/20
22/22 [==============================] - 14s 626ms/step - loss: 2.3203
Epoch 6/20
22/22 [==============================] - 14s 632ms/step - loss: 2.2086
Epoch 7/20
22/22 [==============================] - 14s 628ms/step - loss: 2.1041
Epoch 8/20
22/22 [==============================] - 14s 628ms/step - loss: 1.9979
Epoch 9/20
22/22 [==============================] - 14s 626ms/step - loss: 1.8873
Epoch 10/20
22/22 [==============================] - 14s 628ms/step - loss: 1.7553
Epoch 11/20
22/22 [==============================] - 14s 630ms/step - loss: 1.6391
Epoch 12/20
22/22 [==============================] - 14s 635ms/step - loss: 1.4774
Epoch 13/20
2

## Predictions!

In [42]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(inputs=input_ids, states=states,
                                              return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [56]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [1]:
def predict_next(num_chars=300, start='NTNU'):
    states = None
    next_char = tf.constant([start])
    result = [next_char]
    
    for n in range(num_chars):
        next_char, states = one_step_model.generate_one_step(next_char, states=states)
        result.append(next_char)

    result = tf.strings.join(result)
    print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

In [2]:
predict_next()

NameError: name 'tf' is not defined